In [1]:
# coding: utf-8
from numpy import *
import GPy
from random import sample, randint, random
from sklearn.metrics import roc_auc_score

CNfeats=genfromtxt("./Desktop/tcga/new_CNendo.csv",delimiter=",") #from GISTIC_tcga_endo
CNfeats=CNfeats[1:,:]
CNclasses=genfromtxt("./Desktop/tcga/new_hist_for_GPy_endo.csv",delimiter=",") #TCGA_histolog
CNclasses=CNclasses[1:]


CNclasses=CNclasses.reshape(len(CNclasses),1)
CNclasses[(where(CNclasses==3))]=0
ser_inds=(where(CNclasses==0)[0])
endo_inds=(where(CNclasses==1)[0])
unknown_inds=(where(CNclasses==2)[0])
test_ser_inds=sample(ser_inds,50)
test_endo_inds=sample(endo_inds,50)
train_ser_inds=sample(ser_inds,50)
train_endo_inds=sample(endo_inds,50)
#this up here is kinda wrong D:
print "This part doneee."

This part doneee.


In [2]:
CNclasses.shape

(539L, 1L)

In [3]:
#UCEC CN-based GP classifier
mCN_endo=GPy.models.GPClassification(CNfeats[train_ser_inds+train_endo_inds,:],CNclasses[train_ser_inds+train_endo_inds])
mCN_endo.optimize()
cnendo_pred=mCN_endo.predict(CNfeats[test_ser_inds+test_endo_inds,:])
y2_cnendo=CNclasses[test_ser_inds+test_endo_inds]
round_pred=array([[round(x)] for x in cnendo_pred[0]])
sum(abs(y2_cnendo-round_pred))
roc_auc_score(y2_cnendo, cnendo_pred[0])
# 0.97

0.94320000000000004

In [6]:
(CNclasses).shape


(539L, 1L)

In [17]:
#instead of equal class proportion, half of the original sample set
half_ser_inds=sample(ser_inds,50)
half_endo_inds=sample(endo_inds,200)
halfm=GPy.models.GPClassification(CNfeats[half_ser_inds+half_endo_inds,:],CNclasses[half_ser_inds+half_endo_inds])
halfm.optimize()
otherh_ser=[i for i in ser_inds if i not in half_ser_inds]
otherh_endo=[i for i in endo_inds if i not in half_endo_inds]
halfpred=halfm.predict(CNfeats[otherh_ser+otherh_endo,:])
halfy2=CNclasses[otherh_ser+otherh_endo]
roc_auc_score(halfy2,halfpred[0])
#better with 50% of each class than same class sizes (200-50 vs 50-50)
#

0.9065343258891646

In [29]:
#applying the UCEC CN classifier on CL data
CL_endo=genfromtxt("./Desktop/tcga/new_CLendo.csv",delimiter=",")
CL_endo=CL_endo[1:,:]
clpred_mCN_endo=mCN_endo.predict(CL_endo)

In [ ]:
#exporting the result for R analysis
savetxt("./Desktop/tcga/clpred_UCEC_CN_May18.csv",append(["score"],clpred_mCN_endo[0]),delimiter=",")

In [21]:
clpred_mCN_endo[0].shape

(28L, 1L)

In [ ]:
#brcaexp over matchedbrca? when no cl?
brcaexp=genfromtxt("./Desktop/tcga/brcaexp_prepped.csv", delimiter=",") #tu

rec_stat=genfromtxt("./Desktop/tcga/tripleNeg_class.csv",delimiter=",")

rec_stat=array([[i] for i in rec_stat])
nan_inds=where(isnan(rec_stat))[0]
rec_stat[nan_inds]=0

tn_inds=where(rec_stat==1)[0]
nontn_inds=where(rec_stat==0)[0]
tn_training=sample((tn_inds),30)
tn_testing=[i for i in tn_inds if i not in tn_training  ]
nontn_training=sample(nontn_inds,30)
nontn_testing=[i for i in nontn_inds if i not in nontn_training]

mXbrca=GPy.models.GPClassification(X=transpose(brcaexp[1:,tn_training+nontn_training]),Y=rec_stat[tn_training+nontn_training])
#why does optimization do nothing otherwise???
mXbrca.rbf.variance=10
mXbrca.rbf.lengthscale=10
print mXbrca
mXbrca.optimize()
print mXbrca
#mXbrca predictions on test set
pred=mXbrca.predict(transpose(brcaexp[1:,tn_testing]))
pred2=mXbrca.predict(transpose(brcaexp[1:,nontn_testing]))
fullpred=mXbrca.predict(transpose(brcaexp[1:,tn_testing+nontn_testing]))
fully2=rec_stat[tn_testing+nontn_testing]
roc_auc_score(fully2, fullpred[0])
#0.92
sum([round(x) for x in pred[0]])
sum([round(x) for x in pred2[0]])
sum([abs(fully2[i]-round(fullpred[0][i])) for i in range(len(fully2)) ])
map(len,[tn_training, nontn_training, tn_testing, nontn_testing])
#pretty good
#?do for halfpred? like with endo



#Brca exp
ccle=genfromtxt("./Desktop/tcga/ccle_prepped.csv",delimiter=",") #tu
ccle=ccle[1:,2:]
nonmatched=genfromtxt("./Desktop/tcga/nonmtaching_inds_in_brcatcgaexp_de_ccle.csv", delimiter=",")
matching=array([i for i in range(1170) if i not in nonmatched[1:]])
matchedbrca=brcaexp[matching,] #tu
matchedbrca=matchedbrca[1:,]
training_indsb=sample(range(1,1080),500)
testing_indsb=[i for i in range(1,1080) if i not in training_indsb]

#GPmodel_brca_xp_TN_def=mXbrca
#30 + 30 for tn and nontn
matchedXbrca=GPy.models.GPClassification(X=transpose(matchedbrca[:,tn_training+nontn_training]),Y=rec_stat[tn_training+nontn_training])
matchedXbrca.rbf.lengthscale=10
matchedXbrca.rbf.variance=10
matchedXbrca.optimize()
pred_matchedXbrca=matchedXbrca.predict(transpose(matchedbrca[:,tn_testing+nontn_testing]))
roc_auc_score(rec_stat[tn_testing+nontn_testing],pred_matchedXbrca[0])
#0.92

cl_pred_brX=matchedXbrca.predict(transpose(ccle))
cl_pred_brX[0]

#savetxt("GPC_TN_clpred_exp_9Apr.csv",cl_pred[0],delimiter=",")
#yay, actual res. strings as nans complicated before

In [10]:
#Brca cn
ccle_cn_br=genfromtxt("./Desktop/tcga/ccle_cn_breast.csv",delimiter=",")
brcacn=genfromtxt("./Desktop/tcga/brca_CN_moreprepped.csv",delimiter=",")
TNclass=genfromtxt("./Desktop/tcga/TN_classes_for_brcaCN.csv",delimiter=",")
TNclass=TNclass[1:]
brcacn=brcacn[1:,:]
ccle_cn_br=ccle_cn_br[1:,2:]
TNclass=array([[i] for i in TNclass])
naninds=where(isnan(TNclass))[0]
TNclass[naninds]=0
tninds=where(TNclass==1)[0]
nontninds=where(TNclass==0)[0]
tntrain=sample(tninds,30)
tntest=[i for i in tninds if i not in tntrain]
nontntrain=sample(nontninds,30)
nontntest=[i for i in nontninds if i not in nontntrain]
#30+30 for tn and nontn
GPmodel_brca_cn=GPy.models.GPClassification(transpose(brcacn[:,tntrain+nontntrain]), TNclass[tntrain+nontntrain])
GPmodel_brca_cn.optimize() #setting var/l to 10 not needed here. why???
print GPmodel_brca_cn
predbrcacn1=GPmodel_brca_cn.predict(transpose(brcacn[:,tntest])) #34 len
predbrcacn2=GPmodel_brca_cn.predict(transpose(brcacn[:,nontntest])) #968
sum([round(x) for x in predbrcacn1[0]]) #12
sum([round(x) for x in predbrcacn2[0]]) #338
predbrcacnjoin=GPmodel_brca_cn.predict(transpose(brcacn[:,tntest+nontntest]))
roc_auc_score(TNclass[tntest+nontntest],predbrcacnjoin[0]) #0.53
#roc_auc_score doesn't work with y having only 1 class duh
#roc_auc_score(TNclass[tntest],predbrcacn1[0])
#roc_auc_score(TNclass[nontntest],predbrcacn2[0])
#seems pretty bad
cl_pred_brcacn=GPmodel_brca_cn.predict(transpose(ccle_cn_br))
cl_pred_brcacn[0]
#huge majority close to 0.5
#savetxt("GPC_TN_predicted_cn_11Apr.csv",predbrcacnjoin[0],delimiter=",")
#savetxt("GPC_TN_actual_cn_11Apr.csv",TNclass[tntest+nontntest],delimiter=",")
#savetxt("cl_pred_brca_cn_11Apr.csv",cl_pred_brcacn[0],delimiter=",")


#endo

#Endo cn
#?why CNendo again? because earlier wasn't matched to CLs?
Endo_codes=genfromtxt("./Desktop/tcga/Endo_barcode_hist.csv",delimiter=",",skip_header=1)

#error, but ok with 'skip_header', 'skiprows' no 
Endo_hist=genfromtxt("./Desktop/tcga/graded_hist.csv",delimiter=",")
Endo_CN_mostvar=genfromtxt("./Desktop/tcga/grefined_tcga_endo.csv",delimiter=",")
ccle_cn_endo=genfromtxt("./Desktop/tcga/GISTIC_ccle_cn_endo.csv",delimiter=",")

endocn=Endo_CN_mostvar[1:,3:]
ccle_cn_endo=ccle_cn_endo[1:,2:]
endohist=Endo_hist[1:]

serinds=where(endohist==1)[0]
endoinds=where(endohist==0)[0]

sertrain=sample(serinds,50)
sertest=[i for i in serinds if i not in sertrain]
endotrain=sample(endoinds,50)
endotest=[i for i in endoinds if i not in endotrain]
endohist=array([[i] for i in endohist])
mCN_endo=GPy.models.GPClassification(transpose(endocn[:,endotrain+sertrain]), endohist[endotrain+sertrain])
mCN_endo.optimize()
#again manually changing var/l not needed
GPmodel_endo_cn=mCN_endo
pred_endocn=GPmodel_endo_cn.predict(transpose(endocn[:,endotest+sertest]))
#roc_auc_score(halfy2,halfpred[0]) #0.93
roc_auc_score(endohist[endotest+sertest],pred_endocn[0])
#0.91
cl_pred_endo_cn=GPmodel_endo_cn.predict(transpose(ccle_cn_endo))
cl_pred_endo_cn[0]
#meaningful ^^
[round(x) for x in cl_pred_endo_cn[0]]
#savetxt("cl_pred_endo_cn11Apr.csv",cl_pred_endo_cn[0] ,delimiter=",")
#savetxt("GPC_Endo_cn_actual_11Apr.csv",endohist[endotest+sertest],delimiter=",")
#savetxt("GPC_endo_cn_predicted_11Apr.csv",pred_endocn[0],delimiter=",")


#Endo xp
#doesn't optimize!
endo_exp=genfromtxt("./Desktop/tcga/Endo_exp_mostvar.csv", delimiter=",")
endohist_ref=genfromtxt("./Desktop/tcga/Endo_hist_refined.csv",delimiter=",")
endo_exp=endo_exp[1:,1:]
endohist_ref=endohist_ref[1:]
endohist_ref=endohist_ref.reshape(176,1)

xser_inds=where(endohist_ref==1)[0]
xend_inds=where(endohist_ref==0)[0]
xser_train=sample(xser_inds, 25)
xser_test=[i for i in xser_inds if i not in xser_train]
xend_train=sample(xend_inds, 25)
xend_test=[i for i in xend_inds if i not in xend_train]

mX_endo=GPy.models.GPClassification(transpose(endo_exp[:,xser_train+xend_train]), endohist_ref[xser_train+xend_train])
mX_endo.rbf.variance=10
mX_endo.rbf.lengthscale=10
mX_endo.optimize()
print mX_endo

#isustibog
endoxp=endo_exp[:]
endoxp=endoxp/1000
m2X_endo=GPy.models.GPClassification(transpose(endoxp[:,xser_train+xend_train]), endohist_ref[xser_train+xend_train])
pred_m2Xendo=m2X_endo.predict(transpose(endoxp[:,xser_test+xend_test]))
roc_auc_score(endohist_ref[xser_test+xend_test], pred_m2Xendo[0])
#0.93!


#refined endocn for TCGA match
endocn_ref=genfromtxt("./Desktop/tcga/Endo_CN_mostvar_refined.csv",delimiter=",")
endocn_ref=endocn_ref[1:,3:]

#inds from same endohist_refined as previous endoX

mCNr_endo=GPy.models.GPClassification(transpose(endocn_ref[:,xser_train+xend_train]), endohist_ref[xser_train+xend_train])
pred_endocn_ref=mCNr_endo.predict(transpose(endocn_ref[:,xser_test+xend_test]))
roc_auc_score(endohist_ref[xser_test+xend_test], pred_endocn_ref[0])
#0.89


#integr
#just joining at first
joined=concatenate(((endocn_ref),(endoxp)))
mj=GPy.models.GPClassification(transpose(joined[:,:][:,xser_train+xend_train]), endohist_ref[xser_train+xend_train])
mj.rbf.lengthscale=100
mj.rbf.variance=100
mj.optimize()

pred_endo_joined=mj.predict(transpose(joined[:,:][:,xser_test+xend_test]))
roc_auc_score(endohist_ref[xser_test+xend_test],pred_endo_joined[0])
#0.93

#just join for brca

#some basic combopicking for int

#slightly refined brcaexp auc=0.67


#??? chosen sample sizes


Name : gp_classification
Objective : 59.9084083105
Number of Parameters : 2
Number of Optimization Parameters : 2
Updates : True
Parameters:
  gp_classification.  |  value  |  constraints  |  priors
  rbf.variance        |   10.0  |      +ve      |        
  rbf.lengthscale     |   10.0  |      +ve      |        

Name : gp_classification
Objective : 32.6192144467
Number of Parameters : 2
Number of Optimization Parameters : 2
Updates : True
Parameters:
  gp_classification.  |          value  |  constraints  |  priors
  rbf.variance        |  11.2735959506  |      +ve      |        
  rbf.lengthscale     |  62.6023009046  |      +ve      |        

Name : gp_classification
Objective : 41.8957450394
Number of Parameters : 2
Number of Optimization Parameters : 2
Updates : True
Parameters:
  gp_classification.  |           value  |  constraints  |  priors
  rbf.variance        |  0.922497858191  |      +ve      |        
  rbf.lengthscale     |   1.31844410079  |      +ve      |        



0.86857876712328763